In [44]:
import os
from pathlib import Path
from PIL import Image
import pytesseract
from pdf2image import convert_from_path
import doctr
from doctr.io import DocumentFile
from doctr.models import ocr_predictor
import easyocr
import numpy as np
import pandas as pd
from pathlib import Path
from Levenshtein import distance as levenshtein_distance
import PyPDF2
from datetime import datetime

# Geben Sie den Pfad zu den Poppler-Binaries an
poppler_path = r'C:\poppler-24.08.0\Library\bin'

# Set the TESSDATA_PREFIX environment variable
os.environ['TESSDATA_PREFIX'] = r'C:\Program Files\Tesseract-OCR\tessdata'

# Geben Sie den Pfad zur Tesseract-Ausführungsdatei an
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Initialisiere den OCR Predictor von docTR
ocr_model = ocr_predictor('db_resnet50', 'master', pretrained=True)

# Initialisiere den EasyOCR-Reader für die deutsche Sprache
reader = easyocr.Reader(['de'])


C:\Users\O6819\Entwicklung\Transferlesitung6\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\O6819\Entwicklung\Transferlesitung6\.venv\Lib\site-packages\doctr\models\utils\pytorch.py:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user vi

In [45]:
# Definieren der Verzeichnisse für Eingabe, Ground Truth und Ausgaben
# Hinweis: Die Verzeichnisse müssen an Ihre Umgebung angepasst werden

# Eingabeverzeichnis für die Verarbeitung
input_folder = Path("C:/Users/Transferleistung6/Input_PDF")

# Referenzverzeichnis mit Ground Truth zum Testen
reference_folder = "C:/UsersTransferleistung6/Ground Truth_txt"

# Erstes Ausgabeverzeichnis zum Speichern der Ergebnisse: Tesseract
output_folder_Tesseract_txt = "C:/Users/Transferleistung6/Output_OCR_txt_Tesseract"

# Zweites Ausgabeverzeichnis zum Speichern der Ergebnisse: docTR
output_folder_docTR_txt = "C:/Users/Transferleistung6/Output_OCT_txt_docTR"

# Drittes Ausgabeverzeichnis zum Speichern der Ergebnisse: easyocr
output_folder_easyocr_txt = "C:/Users/Transferleistung6/Output_OCR_txt_EasyOCR"

Verarbeitung von PDF-Dateien mit Tesseract:

In [48]:
# Iterate over all PDF files in the directory
files = input_folder.rglob('*.pdf')

for file in files:
    # Convert PDF pages to images
    images = convert_from_path(file, poppler_path=poppler_path)
    extracted_text = ""

    for image in images:
        # Extract text from each image with German language setting
        extracted_text += pytesseract.image_to_string(image, lang='deu') + "\n"

    # Save the extracted text to a .txt file
    output_file = Path(output_folder_Tesseract_txt) / file.name.replace('.pdf', '.txt')
    with open(output_file, 'w', encoding='utf-8') as text_file:
        text_file.write(extracted_text)

    print(f"Extracted text saved in {output_file}")

KeyboardInterrupt: 

Verarbeitung von PDF-Dateien mit docTR:

In [79]:
# Iteriere über alle PDF-Dateien im Verzeichnis
files = input_folder.rglob('*.pdf')

for file in files:
    # Lade das PDF-Dokument mit docTR
    doc = DocumentFile.from_pdf(file)

    # Verwende das OCR-Modell von docTR, um den Text zu extrahieren
    result = ocr_model(doc)

    # Extrahiere den Text aus dem Ergebnis
    extracted_text = ""
    for page in result.pages:
        for block in page.blocks:
            for line in block.lines:
                for word in line.words:
                    extracted_text += word.value + " "
                extracted_text += "\n"
            extracted_text += "\n"
        extracted_text += "\n"

    # Speichere den extrahierten Text in einer .txt-Datei
    output_file = Path(output_folder_docTR_txt) / file.name.replace('.pdf', '.txt')
    with open(output_file, 'w', encoding='utf-8') as text_file:
        text_file.write(extracted_text)

    print(f"Extrahierter Text gespeichert in {output_file}")

Extrahierter Text gespeichert in C:\Users\O6819\Transferleistung6\Output_OCT_txt_docTR\A.txt
Extrahierter Text gespeichert in C:\Users\O6819\Transferleistung6\Output_OCT_txt_docTR\B.txt
Extrahierter Text gespeichert in C:\Users\O6819\Transferleistung6\Output_OCT_txt_docTR\C.txt
Extrahierter Text gespeichert in C:\Users\O6819\Transferleistung6\Output_OCT_txt_docTR\D.txt
Extrahierter Text gespeichert in C:\Users\O6819\Transferleistung6\Output_OCT_txt_docTR\E.txt
Extrahierter Text gespeichert in C:\Users\O6819\Transferleistung6\Output_OCT_txt_docTR\F.txt
Extrahierter Text gespeichert in C:\Users\O6819\Transferleistung6\Output_OCT_txt_docTR\G.txt
Extrahierter Text gespeichert in C:\Users\O6819\Transferleistung6\Output_OCT_txt_docTR\H.txt
Extrahierter Text gespeichert in C:\Users\O6819\Transferleistung6\Output_OCT_txt_docTR\I.txt
Extrahierter Text gespeichert in C:\Users\O6819\Transferleistung6\Output_OCT_txt_docTR\J.txt
Extrahierter Text gespeichert in C:\Users\O6819\Transferleistung6\Outp

Verarbeitung von PDF-Dateien mit EasyOCR:

In [42]:
# Iteriere über alle PDF-Dateien im Verzeichnis
files = input_folder.rglob('*.pdf')

for file in files:
    # Konvertiere PDF-Seiten in Bilder
    images = convert_from_path(file, poppler_path=poppler_path)
    extracted_text = ""

    for image in images:
        # Extrahiere Text aus jedem Bild mit EasyOCR
        result = reader.readtext(np.array(image), detail=0)
        extracted_text += " ".join(result) + "\n"

    # Speichere den extrahierten Text in einer .txt-Datei
    output_file = Path(output_folder_easyocr_txt) / file.name.replace('.pdf', '.txt')
    with open(output_file, 'w', encoding='utf-8') as text_file:
        text_file.write(extracted_text)

    print(f"Extrahierter Text gespeichert in {output_file}")

Extrahierter Text gespeichert in C:\Users\O6819\Transferleistung6\Output_OCR_txt_EasyOCR\A.txt
Extrahierter Text gespeichert in C:\Users\O6819\Transferleistung6\Output_OCR_txt_EasyOCR\B.txt
Extrahierter Text gespeichert in C:\Users\O6819\Transferleistung6\Output_OCR_txt_EasyOCR\C.txt
Extrahierter Text gespeichert in C:\Users\O6819\Transferleistung6\Output_OCR_txt_EasyOCR\D.txt
Extrahierter Text gespeichert in C:\Users\O6819\Transferleistung6\Output_OCR_txt_EasyOCR\E.txt
Extrahierter Text gespeichert in C:\Users\O6819\Transferleistung6\Output_OCR_txt_EasyOCR\F.txt
Extrahierter Text gespeichert in C:\Users\O6819\Transferleistung6\Output_OCR_txt_EasyOCR\G.txt
Extrahierter Text gespeichert in C:\Users\O6819\Transferleistung6\Output_OCR_txt_EasyOCR\H.txt
Extrahierter Text gespeichert in C:\Users\O6819\Transferleistung6\Output_OCR_txt_EasyOCR\I.txt
Extrahierter Text gespeichert in C:\Users\O6819\Transferleistung6\Output_OCR_txt_EasyOCR\J.txt
Extrahierter Text gespeichert in C:\Users\O6819\Tr

Bechnung der durchschnittlichen und Standardabweichung der Confidence-Werte für Tesseract

Dieser Code verarbeitet PDF-Dateien, extrahiert Text mit Tesseract und berechnet den durchschnittlichen Confidence-Wert für jedes Dokument. Am Ende werden der Gesamtdurchschnitt und die Standardabweichung der Confidence-Werte über alle Dokumente berechnet und ausgegeben.

In [56]:
import pytesseract
from pdf2image import convert_from_path
from pathlib import Path
import numpy as np

# Define the input folder and poppler path
input_folder = Path("C:/Users/Transferleistung6/Input_PDF")
poppler_path = r'C:\poppler-24.08.0\Library\bin'

def calculate_tesseract_confidence(file_path, poppler_path):
    images = convert_from_path(file_path, poppler_path=poppler_path)
    file_confidences = []

    for image in images:
        data = pytesseract.image_to_data(image, lang='deu', output_type=pytesseract.Output.DICT)
        confidences = [int(conf) for conf in data['conf'] if conf != -1]
        file_confidences.extend(confidences)

    if file_confidences:
        average_confidence = np.mean(file_confidences)
    else:
        average_confidence = 0

    return average_confidence, file_confidences

# List to store average confidence values for each document
confidence_values = []

# Iterate over all PDF files in the directory
files = input_folder.rglob('*.pdf')

for file in files:
    # Calculate the average confidence for the document
    average_confidence, confidences = calculate_tesseract_confidence(file, poppler_path)
    confidence_values.append(average_confidence)

    # Print the confidence values for the file
    print(f"Average confidence for {file.name}: {average_confidence:.2f}")

# Calculate and print the overall average and standard deviation
if confidence_values:
    overall_average = np.mean(confidence_values)
    overall_std_dev = np.std(confidence_values)
    print(f"\nGesamtdurchschnittlicher Confidence-Wert: {overall_average:.2f}")
    print(f"Gesamtstandardabweichung des Confidence-Werts: {overall_std_dev:.2f}")
else:
    print("\nKeine Dokumente gefunden oder keine Confidence-Werte berechnet.")

Average confidence for A.pdf: 90.61
Average confidence for B.pdf: 86.78
Average confidence for C.pdf: 83.36
Average confidence for D.pdf: 85.73
Average confidence for E.pdf: 83.56
Average confidence for F.pdf: 82.90
Average confidence for G.pdf: 73.50
Average confidence for H.pdf: 73.17
Average confidence for I.pdf: 65.60
Average confidence for J.pdf: 81.49
Average confidence for K.pdf: 86.13
Average confidence for L.pdf: 86.93
Average confidence for M.pdf: 84.58
Average confidence for N.pdf: 87.30
Average confidence for O.pdf: 86.87

Gesamtdurchschnittlicher Confidence-Wert: 82.57
Gesamtstandardabweichung des Confidence-Werts: 6.48


Bechnung der durchschnittlichen und Standardabweichung der Confidence-Werte für EasyOCR

Dieser Code verarbeitet PDF-Dateien, extrahiert Text mit EasyOCR und berechnet den durchschnittlichen Confidence-Wert für jedes Dokument. Am Ende werden der Gesamtdurchschnitt und die Standardabweichung der Confidence-Werte über alle Dokumente berechnet und ausgegeben.

In [48]:
from pathlib import Path
import numpy as np
from pdf2image import convert_from_path
import easyocr

# Define the input folder and poppler path
input_folder = Path("C:/Users/O6819/Transferleistung6/Input_PDF")
poppler_path = r'C:\poppler-24.08.0\Library\bin'

# Create the reader for EasyOCR
reader = easyocr.Reader(['de'])

# List to store the average confidence values for each document
confidence_values = []

# Iterate over all PDF files in the directory
files = input_folder.rglob('*.pdf')

for file in files:
    # Convert PDF pages to images
    images = convert_from_path(file, poppler_path=poppler_path)
    total_confidence = 0
    total_words = 0

    for image in images:
        # Extract text from each image with EasyOCR and get the confidence value
        results = reader.readtext(np.array(image), detail=1)

        for result in results:
            _, _, confidence = result
            total_confidence += confidence
            total_words += 1

    if total_words > 0:
        average_confidence = total_confidence / total_words
    else:
        average_confidence = 0

    # Store the average confidence value for the document
    confidence_values.append(average_confidence)

    # Print the average confidence value for the file
    print(f"Durchschnittlicher Confidence-Wert für {file.name}: {average_confidence:.2f}")

# Calculate and print the overall average and standard deviation
if confidence_values:
    overall_average = np.mean(confidence_values)
    overall_std_dev = np.std(confidence_values)
    print(f"\nGesamtdurchschnittlicher Confidence-Wert: {overall_average:.2f}")
    print(f"Gesamtstandardabweichung des Confidence-Werts: {overall_std_dev:.2f}")
else:
    print("\nKeine Dokumente gefunden oder keine Confidence-Werte berechnet.")

Durchschnittlicher Confidence-Wert für A.pdf: 0.88
Durchschnittlicher Confidence-Wert für B.pdf: 0.62
Durchschnittlicher Confidence-Wert für C.pdf: 0.81
Durchschnittlicher Confidence-Wert für D.pdf: 0.87
Durchschnittlicher Confidence-Wert für E.pdf: 0.85
Durchschnittlicher Confidence-Wert für F.pdf: 0.79
Durchschnittlicher Confidence-Wert für G.pdf: 0.66
Durchschnittlicher Confidence-Wert für H.pdf: 0.69
Durchschnittlicher Confidence-Wert für I.pdf: 0.47
Durchschnittlicher Confidence-Wert für J.pdf: 0.81
Durchschnittlicher Confidence-Wert für K.pdf: 0.84
Durchschnittlicher Confidence-Wert für L.pdf: 0.80
Durchschnittlicher Confidence-Wert für M.pdf: 0.85
Durchschnittlicher Confidence-Wert für N.pdf: 0.87
Durchschnittlicher Confidence-Wert für O.pdf: 0.82

Gesamtdurchschnittlicher Confidence-Wert: 0.77
Gesamtstandardabweichung des Confidence-Werts: 0.11


Bechnung der durchschnittlichen und Standardabweichung der Confidence-Werte für DocTR

Dieser Code verarbeitet PDF-Dateien, extrahiert Text mit DocTR und berechnet den durchschnittlichen Confidence-Wert für jedes Dokument. Am Ende werden der Gesamtdurchschnitt und die Standardabweichung der Confidence-Werte über alle Dokumente berechnet und ausgegeben.

In [59]:
from pathlib import Path
import numpy as np
import doctr
from doctr.io import DocumentFile
from doctr.models import ocr_predictor

# Define the input folder
input_folder = Path("C:/Users/O6819/Transferleistung6/Input_PDF")

def calculate_docTR_confidence(result):

    all_confidences = [word.confidence for page in result.pages for block in page.blocks for line in block.lines for word in line.words]
    return np.mean(all_confidences) if all_confidences else 0

# List to store the average confidence values for each document
confidence_values = []

# Iterate over all PDF files in the directory
files = input_folder.rglob('*.pdf')

for file in files:
    # Load the PDF document with docTR
    doc = DocumentFile.from_pdf(file)

    # Use the OCR model from docTR to extract text
    result = ocr_model(doc)

    # Calculate the average confidence for the document
    average_confidence = calculate_docTR_confidence(result)
    confidence_values.append(average_confidence)

    # Print the average confidence value for the file
    print(f"Durchschnittlicher Confidence-Wert für {file.name}: {average_confidence:.2f}")

# Calculate and print the overall average and standard deviation
if confidence_values:
    overall_average = np.mean(confidence_values)
    overall_std_dev = np.std(confidence_values)
    print(f"\nGesamtdurchschnittlicher Confidence-Wert: {overall_average:.2f}")
    print(f"Gesamtstandardabweichung des Confidence-Werts: {overall_std_dev:.2f}")
else:
    print("\nKeine Dokumente gefunden oder keine Confidence-Werte berechnet.")

Durchschnittlicher Confidence-Wert für A.pdf: 0.93
Durchschnittlicher Confidence-Wert für B.pdf: 0.92
Durchschnittlicher Confidence-Wert für C.pdf: 0.90
Durchschnittlicher Confidence-Wert für D.pdf: 0.93
Durchschnittlicher Confidence-Wert für E.pdf: 0.91
Durchschnittlicher Confidence-Wert für F.pdf: 0.92
Durchschnittlicher Confidence-Wert für G.pdf: 0.88
Durchschnittlicher Confidence-Wert für H.pdf: 0.90
Durchschnittlicher Confidence-Wert für I.pdf: 0.88
Durchschnittlicher Confidence-Wert für J.pdf: 0.89
Durchschnittlicher Confidence-Wert für K.pdf: 0.92
Durchschnittlicher Confidence-Wert für L.pdf: 0.93
Durchschnittlicher Confidence-Wert für M.pdf: 0.92
Durchschnittlicher Confidence-Wert für N.pdf: 0.92
Durchschnittlicher Confidence-Wert für O.pdf: 0.92

Gesamtdurchschnittlicher Confidence-Wert: 0.91
Gesamtstandardabweichung des Confidence-Werts: 0.02


Berechnung der CER, WER und MER Fehlerquoten basierend auf Morris et al. (2004). Dieser Code nimmt einen Referenzordner, der die Ground Truth enthält, und einen Ausgabeordner, der den Output eines OCR-Anbieters enthält. Die Fehlerquoten WER (Word Error Rate), CER (Character Error Rate) und MER (Match Error Rate) werden für jede Datei berechnet und ausgegeben. Am Ende werden der Gesamtdurchschnitt und die Standardabweichung der Fehlerquoten über alle Dateien berechnet und ausgegeben.

In [78]:
import os
from pathlib import Path
import jiwer
from jiwer import wer, cer, mer
import numpy as np

def compare_ocr_results(ground_truth_folder, ocr_results_folder):
    ground_truth_folder = Path(ground_truth_folder)
    ocr_results_folder = Path(ocr_results_folder)

    wer_values = []
    cer_values = []
    mer_values = []

    for gt_file in ground_truth_folder.glob("*.txt"):
        ocr_file = ocr_results_folder / gt_file.name
        if ocr_file.exists():
            with open(gt_file, 'r', encoding='utf-8') as gt, open(ocr_file, 'r', encoding='utf-8') as ocr:
                ground_truth = gt.read()
                ocr_result = ocr.read()

                error_wer = wer(ground_truth, ocr_result) * 100
                error_cer = cer(ground_truth, ocr_result) * 100
                error_mer = mer(ground_truth, ocr_result) * 100

                wer_values.append(error_wer)
                cer_values.append(error_cer)
                mer_values.append(error_mer)

                print(f"File: {gt_file.name}")
                print(f"Character Error Rate (CER): {error_cer:.2f}")
                print(f"Word Error Rate (WER): {error_wer:.2f}")
                print(f"Match Error Rate (MER): {error_mer:.2f}")
                print("-" * 40)
        else:
            print(f"File {gt_file.name} not found in OCR results folder.")
            print("-" * 40)

    # Calculate and print the overall average and standard deviation for WER, CER, and MER
    if cer_values:
        overall_cer_avg = np.mean(cer_values)
        overall_cer_std = np.std(cer_values)
        print(f"\nGesamtdurchschnittlicher CER: {overall_cer_avg:.2f}")
        print(f"Gesamtstandardabweichung des CER: {overall_cer_std:.2f}")
    else:
        print("\nKeine CER-Werte berechnet.")

    if wer_values:
        overall_wer_avg = np.mean(wer_values)
        overall_wer_std = np.std(wer_values)
        print(f"\nGesamtdurchschnittlicher WER: {overall_wer_avg:.2f}")
        print(f"Gesamtstandardabweichung des WER: {overall_wer_std:.2f}")
    else:
        print("\nKeine WER-Werte berechnet.")

    if mer_values:
        overall_mer_avg = np.mean(mer_values)
        overall_mer_std = np.std(mer_values)
        print(f"\nGesamtdurchschnittlicher MER: {overall_mer_avg:.2f}")
        print(f"Gesamtstandardabweichung des MER: {overall_mer_std:.2f}")
    else:
        print("\nKeine MER-Werte berechnet.")

# Example usage
ground_truth_folder = "C:/Users/Transferleistung6/Ground Truth_txt"
ocr_results_folder = "C:/Users/ransferleistung6/Output_OCR_txt_EasyOCR"
compare_ocr_results(ground_truth_folder, ocr_results_folder)

File: A.txt
Character Error Rate (CER): 6.78
Word Error Rate (WER): 25.09
Match Error Rate (MER): 23.49
----------------------------------------
File: B.txt
Character Error Rate (CER): 42.07
Word Error Rate (WER): 76.32
Match Error Rate (MER): 49.15
----------------------------------------
File: C.txt
Character Error Rate (CER): 15.00
Word Error Rate (WER): 37.46
Match Error Rate (MER): 35.22
----------------------------------------
File: D.txt
Character Error Rate (CER): 39.21
Word Error Rate (WER): 62.88
Match Error Rate (MER): 51.49
----------------------------------------
File: E.txt
Character Error Rate (CER): 18.79
Word Error Rate (WER): 49.41
Match Error Rate (MER): 43.06
----------------------------------------
File: F.txt
Character Error Rate (CER): 45.12
Word Error Rate (WER): 80.67
Match Error Rate (MER): 56.32
----------------------------------------
File: G.txt
Character Error Rate (CER): 24.06
Word Error Rate (WER): 66.27
Match Error Rate (MER): 62.22
--------------------